# Generic notebook to guide on the conventions for updating a Silver layer Delta Table

### Configuration
|Item|Value|
|---|---|
|Parameter|Expects a parameter with a JSON object that contains the source and target directory details.|
|Source File Identification|If the "RepopulateTableFromSource" parameter is "True", all source files are read as input. If "RepopulateTableFromSource" is False, the latest file available in source directory will be retrieved dynamically|
|Output|It will write the output to a Delta table in the silver database stored in the silver container of the data lake in the directory defined by JSON parameter|
|Manipulations included|Data Type correction, normalise JSON, hard structure changes to enable platinum layer to function efficiently|

# Import all libraries required
Only import the libraries and functions that are actually used.

In [0]:
import json
import pyspark.sql
from pyspark.sql.functions import lit,col,date_format,unix_timestamp, substring, regexp_replace, concat, input_file_name
from pyspark.sql.types import TimestampType

# Call Administration notebook to perform tasks before the data modelling can continue

In [0]:
%run ../Administration/CreateDatabaseIfNotExists

# Call the applicable Helper Functions notebooks to include their functions for use in this Notebook at runtime

In [0]:
%run ../HelperFunctions/DataLakeHelperFunctions

# Parameters / Widgets

Here the single JSON parameter widget of the notebook will be generated.

Having it as JSON means we can send multiple values from the calling code such as Data Factory to the notebook at run time with a single widget / parameter. 

When we want to add more parameter values leter, it doesn't require a notebook change to add more widgets OR a data factory change to add more parameters. Just change the value sent. 

The JSON object will be unpacked and the attributes needed will be extracted in to variables with the "p" prefix to show it was a notebook level parameter.

Run this when you want to re-initialise the widget in the following cell with new default values. 
Do not let this cell run as part of normal operations

In [0]:
#Run this when you want to re-initialise the widget in the following cell with new default values. 
#Do not let this cell run as part of normal operations
#dbutils.widgets.removeAll()

### Define the widget
Note, once createed, it stays attached to the notebook. Keeping this code to run each time to ensure it exists. 
|Parameter Value|Description|
|--|--|
|"SourceDataLakeContainer"|Container where the source files are located|
|"SourceDataLakeDirectory"|Directory where the source files are located, all files pertain to the same entity/table|
|"TargetDataLakeContainer"|Container where the target delta lake table is located|
|"TargetDataLakeDirectory"|Directory in the target container where the target delta table will be stored. One folder above the _delta_log folder of this delta table.|
|"TargetEntityName"|Name of the entity being processed. Will become the name of the target delta table.|
|"RepopulateTableFromSource"|This field can be used to indicate via a **True** value that all source files are to be read, and with a **False** only the latest file is to be read|

In [0]:
# Create the widget in the first place with a default value one can use for testing
# This iteration expects a single JSON Object, not an array
# Note, the IngestionDateTimeStampUTC has to be in the format defined here, or the timestamp conversion later on will not work
dbutils.widgets.text("widgetJSONString", 
'''
{
"SourceDataLakeContainer": "rawdata",
"SourceDataLakeDirectory": "WorldWideImporters/Batch/wwi-dimstockitem/",
"TargetDataLakeContainer": "silver",
"TargetDataLakeDirectory": "WorldWideImporters/Batch/wwi-dimstockitem/",
"TargetEntityName": "wwi_dimstockitem",
"RepopulateTableFromSource": "True"
}
'''
)

### Transform parameter values received into usable format
* Data type conversion
* String manipulation
* Property extraction
* etc.

Currently this focusses on the file path, but will be extended to work on all parameter values received

In [0]:
# At this stage, the string in the variable is still just a string, not typed as JSON. 
# Convert it to a JSON typed value using json.loads
pNotebookWidgetWithJSONString = json.loads(dbutils.widgets.get("widgetJSONString"))

# Print out full value received for logging purposes
print("pNotebookWidgetWithJSONString: " + str(pNotebookWidgetWithJSONString))

# Assign each attribute to the applicable variabe to be used going forward
pSourceDataLakeContainer = pNotebookWidgetWithJSONString["SourceDataLakeContainer"]
pSourceDataLakeDirectory = pNotebookWidgetWithJSONString["SourceDataLakeDirectory"]
pTargetDataLakeContainer = pNotebookWidgetWithJSONString["TargetDataLakeContainer"]
pTargetDataLakeDirectory = pNotebookWidgetWithJSONString["TargetDataLakeDirectory"]
pTargetEntityName = pNotebookWidgetWithJSONString["TargetEntityName"]
pRepopulateTableFromSource = pNotebookWidgetWithJSONString["RepopulateTableFromSource"]


pNotebookWidgetWithJSONString: {'SourceDataLakeContainer': 'rawdata', 'SourceDataLakeDirectory': 'WorldWideImporters/Batch/wwi-dimstockitem/', 'TargetDataLakeContainer': 'silver', 'TargetDataLakeDirectory': 'WorldWideImporters/Batch/wwi-dimstockitem/', 'TargetEntityName': 'wwi_dimstockitem', 'RepopulateTableFromSource': 'True'}

# Ensure applicable data lake containers are mounted

Once this has run once, it should never have to run again. Just including for safety in all notebooks.
* Source and Target containers should be mounted

In [0]:
mount_lake_container(pSourceDataLakeContainer)

Mount Point: /mnt/datalake_rawdata

In [0]:
mount_lake_container(pTargetDataLakeContainer)

Mount Point: /mnt/datalake_silver

# Get the latest file available for the source directory provided

In [0]:
#For the source data lake container and directory, convert it into a usable mount point string value
#Expected output format: /mnt/datalake_<containername>/<Directory>/
vSourceDataDirectoryMountPoint = convert_container_and_directory_to_mountpoint(pSourceDataLakeContainer, pSourceDataLakeDirectory)

#Check values
print('vSourceDataDirectoryMountPoint: ' + vSourceDataDirectoryMountPoint)

#For the given source directory mount point path, get the latest file available
#Expected output format: dbfs:/mnt/datalake_<containername>/<Directory>/<filename>.<extension>
vSourceDataLakeFileFullPath = get_latest_modified_file_from_directory(vSourceDataDirectoryMountPoint)

#Check Values
print('vSourceDataLakeFileFullPath: ' + vSourceDataLakeFileFullPath)

vSourceDataDirectoryMountPoint: /mnt/datalake_rawdata/WorldWideImporters/Batch/wwi-dimstockitem/
vSourceDataLakeFileFullPath: dbfs:/mnt/datalake_rawdata/WorldWideImporters/Batch/wwi-dimstockitem/2022/04/04/11/16/wwi-dimstockitem_20220407_1116.csv

# From the latest available file directory, exctract the source file's ingestion date time stamp into the data lake. 
This will be used for partitioning in the silver delta table

In [0]:
#Starting string example: dbfs:/mnt/datalake_rawdata/WorldWideImporters/Batch/wwi-dimstockitem/2022/03/03/11/47/wwi-dimstockitem.csv

#Remove the full mount point and directory from the full file name, leaving the date time stamp folders and the file name itself
#Output example: dbfs:2022/03/03/11/47/wwi-dimstockitem.csv
vIngestionDateTimeStampUTC = vSourceDataLakeFileFullPath.replace(vSourceDataDirectoryMountPoint, '')

#Remove the dbfs: prefix
#Output example: 2022/03/03/11/47/wwi-dimstockitem.csv
vIngestionDateTimeStampUTC = vIngestionDateTimeStampUTC.replace('dbfs:', '')

#Extract the subsections from the date value
vYear = vIngestionDateTimeStampUTC[0:4]
vMonth = vIngestionDateTimeStampUTC[5:7]
vDay = vIngestionDateTimeStampUTC[8:10]
vHour = vIngestionDateTimeStampUTC[11:13]
vMinute = vIngestionDateTimeStampUTC[14:16]

#Construct the final integer representation of the ingestion date time stamp
vIngestionDateTimeStampUTC = vYear + vMonth + vDay + vHour + vMinute

print(vIngestionDateTimeStampUTC)

202204041116

# Import source data into Data Frame and create temporary views for use in this notebook
* These import functions will be different depending on the source file types and configuration. 
* The standard IngestionDateTimeUTC value is added as a field in the data at this point do that partitioning can be done on the field in the target delta table

In [0]:
def read_latest_file_from_source_into_dataFrame():
    """
    Read data from the latest file identified into a dataframe for manipulation.
    The standard IngestionDateTimeStampUTC field used for partitioning is added here already from the variable defined above
    Will only be called when the "RepopulateTableFromSource" = "False"
    """
    rawSourceDF = spark.read.format("csv")\
    .options(header='true', inferSchema='true', delimiter='|')\
    .load(vSourceDataLakeFileFullPath)\
    .withColumn("IngestionDateTimeStampUTC", lit(vIngestionDateTimeStampUTC))

    # Take the data frame created and generate a temp view for further processing
    rawSourceDF.createOrReplaceTempView("rawSourceDF")
    
    return rawSourceDF

In [0]:
def read_all_files_from_source_into_dataFrame():
    """
    Read data from the source files directory into a dataframe for manipulation.
    The standard IngestionDateTimeStampUTC field used for partitioning is added here already from the parent directory of the files
    Will only be called when the "RepopulateTableFromSource" = "True"
    """

    #Make new version of the source directory to add the dbfs: that is added by the input_file_name() function. 
    vSourceDataLakeDirectoryPathToRemoveFromFullFilePath = 'dbfs:' + vSourceDataDirectoryMountPoint

    #Transformations description
    #FileFullPath: Get the source file name in full e.g. dbfs:/mnt/datalake_rawdata/WorldWideImporters/Batch/wwi-dimstockitem/2022/04/04/11/16/wwi-dimstockitem_20220407_1116.csv
    #FileIngestionTimeStampDirectoriesAndFileName: Remove everything from dbfs: until the year directory so you are left with 2022/04/04/11/16/wwi-dimstockitem_20220407_1116.csv
    #FileIngestionTimeStampDirectories: extract only the section of the directories that are the ingestion time stamp values e.g. 2022/04/04/11/16
    #IngestionDateTimeStampUTC: Remove the / characters in order to get an integer representation 202204041116
    #Then drop fie intermediate fields not required. 
    rawSourceDF = spark.read.format("csv")\
        .options(header='true', inferSchema='true', delimiter='|', recursiveFileLookup='true')\
        .load(vSourceDataLakeDirectory_test)\
        .withColumn("FileFullPath", input_file_name())\
        .withColumn("FileIngestionTimeStampDirectoriesAndFileName", regexp_replace("FileFullPath", vSourceDataLakeDirectoryPathToRemoveFromFullFilePath, ''))\
        .withColumn("FileIngestionTimeStampDirectories", substring("FileIngestionTimeStampDirectoriesAndFileName", 0, 16))\
        .withColumn("IngestionDateTimeStampUTC", regexp_replace("FileIngestionTimeStampDirectories", '/', ''))\
        .drop("FileFullPath", "FileIngestionTimeStampDirectoriesAndFileName", "FileIngestionTimeStampDirectories")
    
    # Take the data frame created and generate a temp view for further processing
    rawSourceDF.createOrReplaceTempView("rawSourceDF")
    
    return rawSourceDF


Based on the RepopulateTableFromSource parameter passed in , either read all source files into a data frame, or only read the latest file. 

During normal operations, pRepopulateTableFromSource should be **False**

In [0]:
if(pRepopulateTableFromSource=='True'):
    rawSourceDF = read_all_files_from_source_into_dataFrame()
else:
    rawSourceDF = read_latest_file_from_source_into_dataFrame()

# Apply transformations

Top 10 records view of the source data for reference

In [0]:
%sql
SELECT *
FROM rawSourceDF
LIMIT 10

Stock Item Key,WWI Stock Item ID,Stock Item,Color,Selling Package,Buying Package,Brand,Size,Lead Time Days,Quantity Per Outer,Is Chiller Stock,Barcode,Tax Rate,Unit Price,Recommended Retail Price,Typical Weight Per Unit,Valid From,Valid To,Lineage Key,IngestionDateTimeStampUTC
0,0,Unknown,N/A,N/A,N/A,N/A,N/A,0,0,false,N/A,0.0,0.0,0.0,0.0,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,0,202203031147
1,219,Void fill 400 L bag (White) 400L,N/A,Each,Each,N/A,400L,14,10,false,N/A,14.0,50.0,74.75,1.0,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,202203031147
2,218,Void fill 300 L bag (White) 300L,N/A,Each,Each,N/A,300L,14,10,false,N/A,14.0,37.5,56.06,0.75,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,202203031147
3,217,Void fill 200 L bag (White) 200L,N/A,Each,Each,N/A,200L,14,10,false,N/A,14.0,25.0,37.38,0.5,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,202203031147
4,216,Void fill 100 L bag (White) 100L,N/A,Each,Each,N/A,100L,14,10,false,N/A,14.0,12.5,18.69,0.25,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,202203031147
5,215,Air cushion machine (Blue),N/A,Each,Each,N/A,N/A,20,1,false,N/A,20.0,1899.0,2839.01,10.0,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,202203031147
6,214,Air cushion film 200mmx200mm 325m,N/A,Each,Each,N/A,325m,14,1,false,N/A,14.0,90.0,134.55,6.0,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,202203031147
7,213,Air cushion film 200mmx100mm 325m,N/A,Each,Each,N/A,325m,14,1,false,N/A,14.0,87.0,130.07,5.0,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,202203031147
8,212,Large replacement blades 18mm,N/A,Each,Each,N/A,18mm,14,10,false,N/A,14.0,4.3,6.43,0.8,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,202203031147
9,211,Small 9mm replacement blades 9mm,N/A,Each,Each,N/A,9mm,14,10,false,N/A,14.0,4.1,6.13,0.7,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,202203031147


* Rename to fit to parquet naming standard
  * No spaces or specical characters. Start with underscore or alphabet letter. 
* Cast to appropriate data type
  * Data type reference: [cast function](https://docs.microsoft.com/en-us/azure/databricks/sql/language-manual/functions/cast)
* Remember to use \` character to encapsulate field names
* Save results to new temp view that is referenced either by the next transformation step, or by the step that writes the output to lake
* Note
  * It is more efficient to perform all new column additions and type transforms in SQL because .withColumn and .withColumnRenamed creates a new dataframe after each call.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW transformedView
AS
SELECT 
    CAST(`Stock Item Key` AS INT ) AS `StockItemKey`
  , CAST(`WWI Stock Item ID` AS INT ) AS `WWIStockItemID`
  , CAST(`Stock Item` AS VARCHAR(8000) ) AS `StockItem`
  , CAST(`Color` AS VARCHAR(8000) ) AS `Color`
  , CAST(`Selling Package` AS VARCHAR(8000) ) AS `SellingPackage`
  , CAST(`Buying Package` AS VARCHAR(8000) ) AS `Buying`
  , CAST(`Brand` AS VARCHAR(8000) ) AS `Brand`
  , CAST(`Size` AS VARCHAR(8000) ) AS `Size`
  , CAST(`Lead Time Days` AS INT ) AS `LeadTimeDays`
  , CAST(`Quantity Per Outer` AS INT ) AS `QuantityPerOuter`
  , CAST(`Is Chiller Stock` AS BOOLEAN ) AS `IsChillerStock`
  , CAST(`Barcode` AS VARCHAR(8000) ) AS `Barcode`
  , CAST(`Tax Rate` AS DECIMAL(19,4) ) AS `TaxRate`
  , CAST(`Unit Price` AS DECIMAL(19,4) ) AS `UnitPrice`
  , CAST(`Recommended Retail Price` AS DECIMAL(19,4) ) AS `RecommendedRetailPrice`
  , CAST(`Typical Weight Per Unit` AS DECIMAL(19,4) ) AS `TypicalWeightPerUnit`
  , CAST(`Valid From` AS TIMESTAMP ) AS `ValidFrom`
  , CAST(`Valid To` AS TIMESTAMP ) AS `ValidTo`
  , CAST(`Lineage Key` AS INT ) AS `LineageKey`
  /*Metadata fields added by the notebook*/
  , `IngestionDateTimeStampUTC` AS `IngestionDateTimeStampUTC`
FROM rawSourceDF

In [0]:
%sql
SELECT * 
FROM transformedView

StockItemKey,WWIStockItemID,StockItem,Color,SellingPackage,Buying,Brand,Size,LeadTimeDays,QuantityPerOuter,IsChillerStock,Barcode,TaxRate,UnitPrice,RecommendedRetailPrice,TypicalWeightPerUnit,ValidFrom,ValidTo,LineageKey,IngestionDateTimeStampUTC
0,0,Unknown,N/A,N/A,N/A,N/A,N/A,0,0,false,N/A,0.0000,0.0000,0.0000,0.0000,2013-01-01T00:00:00.000+0000,9999-12-31T23:59:59.999+0000,0,202203031147
1,219,Void fill 400 L bag (White) 400L,N/A,Each,Each,N/A,400L,14,10,false,N/A,14.0000,50.0000,74.7500,1.0000,2013-01-01T00:00:00.000+0000,2016-05-31T23:00:00.000+0000,5,202203031147
2,218,Void fill 300 L bag (White) 300L,N/A,Each,Each,N/A,300L,14,10,false,N/A,14.0000,37.5000,56.0600,0.7500,2013-01-01T00:00:00.000+0000,2016-05-31T23:00:00.000+0000,5,202203031147
3,217,Void fill 200 L bag (White) 200L,N/A,Each,Each,N/A,200L,14,10,false,N/A,14.0000,25.0000,37.3800,0.5000,2013-01-01T00:00:00.000+0000,2016-05-31T23:00:00.000+0000,5,202203031147
4,216,Void fill 100 L bag (White) 100L,N/A,Each,Each,N/A,100L,14,10,false,N/A,14.0000,12.5000,18.6900,0.2500,2013-01-01T00:00:00.000+0000,2016-05-31T23:00:00.000+0000,5,202203031147
5,215,Air cushion machine (Blue),N/A,Each,Each,N/A,N/A,20,1,false,N/A,20.0000,1899.0000,2839.0100,10.0000,2013-01-01T00:00:00.000+0000,2016-05-31T23:00:00.000+0000,5,202203031147
6,214,Air cushion film 200mmx200mm 325m,N/A,Each,Each,N/A,325m,14,1,false,N/A,14.0000,90.0000,134.5500,6.0000,2013-01-01T00:00:00.000+0000,2016-05-31T23:00:00.000+0000,5,202203031147
7,213,Air cushion film 200mmx100mm 325m,N/A,Each,Each,N/A,325m,14,1,false,N/A,14.0000,87.0000,130.0700,5.0000,2013-01-01T00:00:00.000+0000,2016-05-31T23:00:00.000+0000,5,202203031147
8,212,Large replacement blades 18mm,N/A,Each,Each,N/A,18mm,14,10,false,N/A,14.0000,4.3000,6.4300,0.8000,2013-01-01T00:00:00.000+0000,2016-05-31T23:00:00.000+0000,5,202203031147
9,211,Small 9mm replacement blades 9mm,N/A,Each,Each,N/A,9mm,14,10,false,N/A,14.0000,4.1000,6.1300,0.7000,2013-01-01T00:00:00.000+0000,2016-05-31T23:00:00.000+0000,5,202203031147


# Output data to lake
* See the things to do list at the top of the notebook on how this needs to be automated

### Ensure the target Delta table exists in the expected location. Create if not.

Define the expected target location from the parameters received

In [0]:
#Example values expected
# "pTargetDataLakeContainer": "silver",
# "pTargetDataLakeDirectory": "WorldWideImporters/Batch/wwi-dimstockitem"
vTargetDeltaTablePath = '/mnt/datalake_' + pTargetDataLakeContainer + '/' +pTargetDataLakeDirectory

#Check values
print('vTargetDeltaTablePath: ' + vTargetDeltaTablePath)

vTargetDeltaTablePath: /mnt/datalake_silver/WorldWideImporters/Batch/wwi-dimstockitem/

Dynamic SQL Statement that will only create the table on the first run. 
* If the table already exists, this will do nothing. 
* Default source data is everying in the current source file being processed. 
* Partitioning - Silver Layer Logic
  * **Facts** - Partition by IngestionDateTimeStampUTC only when this data is to be ingested into a fact table with high volume of data to make the insert easier
  * **Dimensions** - Do not partition if this is to be used in a Dimension in the platinum layer because then we read the entire silver entity table, not just the latest data to update the target dimension correctly.

In [0]:
#TargetEntityName": "wwi-dimstockitem",
vDeltaTableCreateStatement = 'CREATE TABLE IF NOT EXISTS ' \
+ 'datalakehouse_' + pTargetDataLakeContainer + '.' + pTargetEntityName + ' \n' \
+ 'USING DELTA ' + '\n' \
+ 'LOCATION \'' + vTargetDeltaTablePath + '\' ' + '\n' \
+ 'PARTITIONED BY ( IngestionDateTimeStampUTC )' + '\n' \
+ 'AS' + '\n' \
+ 'SELECT * FROM transformedView'

#Check final output
print('vDeltaTableCreateStatement: ' + vDeltaTableCreateStatement)

#Exsecute the SQL
spark.sql(vDeltaTableCreateStatement)


vDeltaTableCreateStatement: CREATE TABLE IF NOT EXISTS datalakehouse_silver.wwi_dimstockitem 
USING DELTA 
LOCATION '/mnt/datalake_silver/WorldWideImporters/Batch/wwi-dimstockitem/' 
PARTITIONED BY ( IngestionDateTimeStampUTC )
AS
SELECT * FROM transformedView
Out[97]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

## Merge the incoming source data into the output delta table

* Don't filter on the IngestionDateTimeStampUTC field for dimensions as they are not partitioned.

In [0]:
%sql
/*Since no columns are defined, it auto populates using the field names which should match between source and target*/
MERGE INTO datalakehouse_silver.wwi_dimstockitem as target

USING transformedView as source
  /*Add business keys here for join */
  ON target.StockItemKey = source.StockItemKey
    /*Always join on the ingestion date time as well to ensure you only affect the partition linked to this source file*/
    AND target.IngestionDateTimeStampUTC = source.IngestionDateTimeStampUTC
    
WHEN MATCHED 
  THEN UPDATE SET *
  
WHEN NOT MATCHED 
  THEN INSERT *
;


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2688,672,0,2016


## Perform optimisations to the delta table for better performance

* ZORDER BY: To ensure the business keys of the data you would join or filter on are sorted correctly. Allows for data skipping on read and more efficient joins. 
  * Dont add too many columns in the CSV list, with each column added, the effectiveness of the sorting has less effect. 
    * Only add the keys that will be used in joins to other tables i.e. the business keys
  * **Dimension / List / Lookup / MasterDate** source entities - do on the entire silver table to ensure joining in platinum is efficient. 
    * The WHERE clause will simply be removed. 
  * **Fact / Transactions** source entities - Only do it on the latest data added via the WHERE clause i.e. the current IngestionDateTimeStampUTC - if you zorder older data that was already ordered, it will have no benifit and just 
  * This is a dynamic SQL statement by default to ensure we can filter to the latest partition only to ensure efficiency. 
    * Even though the WHERE clause wont be available for files that will become dimensions, keep it as a dynamic SQL statement for consistency. 
    
Sample query expected for **Dimension / List / Lookup / MasterDate** source entities
```
OPTIMIZE datalakehouse_silver.Product
ZORDER BY `ProductID`
```

Sample query expected for **Fact / Transactions** source entities
```
OPTIMIZE datalakehouse_silver.SalesOrderHeader
WHERE `IngestionDateTimeStampUTC` = 202203071422
ZORDER BY `SalesOrderID`
```

In [0]:
#Create the string that is the SQL query to execute using the latest ingestion time stamp and the right entity name
vOptimiseTableWithZOrderClauseSQL = 'OPTIMIZE datalakehouse_' + pTargetDataLakeContainer + '.' + pTargetEntityName + '\n' \
+ 'WHERE `IngestionDateTimeStampUTC` = ' + vIngestionDateTimeStampUTC + '\n' \
+ 'ZORDER BY `StockItemKey`'

#Check the statement to be executed
print(vOptimiseTableWithZOrderClauseSQL)

#Exsecute the SQL
vOptimiseTableWithZOrderClauseSQL_resultDF = spark.sql(vOptimiseTableWithZOrderClauseSQL)

# Check the output of the dynamic SQL query
display(vOptimiseTableWithZOrderClauseSQL_resultDF)

OPTIMIZE datalakehouse_silver.wwi_dimstockitem
WHERE `IngestionDateTimeStampUTC` = 202204041116
ZORDER BY `StockItemKey`

path,metrics
dbfs:/mnt/datalake_silver/WorldWideImporters/Batch/wwi-dimstockitem,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 1, List(minCubeSize(107374182400), List(0, 0), List(1, 17738), 0, List(0, 0), 0, null), 0, 1, 1, false)"


* VACUUM: Remove previous version files that are no longer needed if older than the specified number of hours. 168 hours = 7 days is the default. 
  * Note, this means timetravel to before this period will not be possible. 
  * This ensures the table remains clean and as small as possible

Sample query expected
```
VACUUM datalakehouse_silver.SalesOrderHeader RETAIN 168 HOURS
```

In [0]:
#Create the string that is the SQL query to execute
vVacuumTableSQLStatement = 'VACUUM datalakehouse_' + pTargetDataLakeContainer + '.' + pTargetEntityName + ' RETAIN 168 HOURS'

#Check the statement to be executed
print(vVacuumTableSQLStatement)

#Exsecute the SQL
vVacuumTableSQLStatement_resultDF = spark.sql(vVacuumTableSQLStatement)

# Check the output of the dynamic SQL query
display(vVacuumTableSQLStatement_resultDF)

VACUUM datalakehouse_silver.wwi_dimstockitem RETAIN 168 HOURS

path
dbfs:/mnt/datalake_silver/WorldWideImporters/Batch/wwi-dimstockitem
